In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install -q flash_attn triton

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab_Notebooks/Session29

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/Session29


In [ ]:

import datasets
from datasets import load_dataset
dataset_name = "OpenAssistant/oasst1"
dataset = load_dataset(dataset_name, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

In [ ]:

#convert the dataset to pandas dataframe
df = dataset.to_pandas()
assistances = df[(df['role'] == 'assistant') & (df['rank'] == 0.0)]
prompters = df[df['role'] == 'prompter']
prompters.set_index('message_id',inplace=True)
assistant_prompt_values = []
for _,row in assistances.iterrows():
  prompt_text = prompters.loc[row.parent_id,'text']
  new_row = "### Human: " + prompt_text + "### Assistant: " + row.text
  assistant_prompt_values.append(new_row)

assistances['prompt_assistance_text'] = assistant_prompt_values
# convert into HG dataset
finetuning_dataset = datasets.Dataset.from_pandas(assistances)
finetuning_dataset['prompt_assistance_text'][:3]


<ipython-input-4-7ad9384e37c7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assistances['prompt_assistance_text'] = assistant_prompt_values


['### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, lea

In [ ]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "microsoft/Phi-3.5-Mini-Instruct"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-Mini-Instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-Mini-Instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_fe

In [ ]:
# Print all the named modules in the model
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.embed_dropout
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.qkv_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.activation_fn
model.layers.0.input_layernorm
model.layers.0.resid_attn_dropout
model.layers.0.resid_mlp_dropout
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.qkv_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.activation_fn
model.layers.1.input_layernorm
model.layers.1.resid_attn_dropout
model.layers.1.resid_mlp_dropout
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.o_proj
model.layers.2.self_attn.qkv_proj
model.layers.2.self_attn.rotary_emb
model.layers.

In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,

    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

In [ ]:


from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    #gradient_checkpointing=True,
)



In [ ]:

from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
    model=model,
    train_dataset=finetuning_dataset,
    peft_config=peft_config,
    dataset_text_field="prompt_assistance_text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/17972 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)
        print(name,module)

base_model.model.model.layers.0.input_layernorm Phi3RMSNorm()
base_model.model.model.layers.0.post_attention_layernorm Phi3RMSNorm()
base_model.model.model.layers.1.input_layernorm Phi3RMSNorm()
base_model.model.model.layers.1.post_attention_layernorm Phi3RMSNorm()
base_model.model.model.layers.2.input_layernorm Phi3RMSNorm()
base_model.model.model.layers.2.post_attention_layernorm Phi3RMSNorm()
base_model.model.model.layers.3.input_layernorm Phi3RMSNorm()
base_model.model.model.layers.3.post_attention_layernorm Phi3RMSNorm()
base_model.model.model.layers.4.input_layernorm Phi3RMSNorm()
base_model.model.model.layers.4.post_attention_layernorm Phi3RMSNorm()
base_model.model.model.layers.5.input_layernorm Phi3RMSNorm()
base_model.model.model.layers.5.post_attention_layernorm Phi3RMSNorm()
base_model.model.model.layers.6.input_layernorm Phi3RMSNorm()
base_model.model.model.layers.6.post_attention_layernorm Phi3RMSNorm()
base_model.model.model.layers.7.input_layernorm Phi3RMSNorm()
base_mo

In [ ]:

trainer.train()
new_model="phi-3.5-mini-oasst1-finetunned"
trainer.model.save_pretrained(new_model)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.429400
20,1.319600
30,1.408800
40,1.628600
50,1.678100
60,1.235200
70,1.213900
80,1.345700
90,1.453300
100,1.724000


In [ ]:

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [ ]:
# Run text generation pipeline
prompt = "What is a model explainability?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] What is a model explainability? [/INST] Model explainability refers to the ability to understand and interpret the decisions made by a machine learning model. It is an important aspect of model development and deployment, as it helps to ensure that the model's decisions are transparent, fair, and accountable. Model explainability can be achieved through various techniques, such as feature importance analysis, decision tree visualization, and rule-based models. These techniques help to identify the factors that contribute to the model's decisions and provide insights into how the model works. Model explainability is particularly important in high-stakes applications, such as healthcare and finance, where the consequences of incorrect or unfair decisions can be severe. By providing transparency and accountability, model explainability helps to build trust in machine learning models and ensure that they are used responsibly.

In summary, model explainability is the ability to under

In [ ]:
# Run text generation pipeline
prompt = "What is microsoft/Phi-3.5-Mini-Instruct"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] What is microsoft/Phi-3.5-Mini-Instruct [/INST] Microsoft Phi-3.5-Mini-Instruct is a project that aims to provide a minimal instruction set for the Phi architecture, which is a hardware architecture developed by Microsoft Research. The Phi architecture is designed to be highly efficient and scalable, making it well-suited for high-performance computing applications. The project provides a set of instructions that can be used to program the Phi architecture, allowing developers to take advantage of its unique features and capabilities.

The Phi architecture is based on a new type of processor called a "field-programmable gate array" (FPGA), which allows for highly customizable and flexible hardware design. The Phi architecture is designed to be highly efficient and scalable, making it well-suited for high-performance computing applications. The Phi architecture is also designed to be highly energy-efficient, making it well-suited for use in data centers and other high-performance

In [ ]:
# Run text generation pipeline
prompt = "Who won the ICC world cup"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] Who won the ICC world cup [/INST] The ICC World Cup is a cricket tournament that is held every four years. The first ICC World Cup was held in 1975, and the most recent one was held in 2019. The 2019 ICC World Cup was won by England, who defeated Australia in the final match. The tournament is considered to be the most prestigious cricket event in the world, and the winner is awarded the ICC World Cup Trophy. The tournament is played over a period of five weeks, with matches taking place in various countries around the world. The ICC World Cup is a major event in the cricket calendar, and it is watched by millions of people around the world. The tournament is also a great opportunity for young players to showcase their skills and for teams to compete at the highest level. The ICC World Cup is a great example of how sports can bring people together and create a sense of unity and pride. The tournament is a great opportunity for people to come together and celebrate the sport of c

In [ ]:
# Run text generation pipeline
prompt = "Who won the ICC world cup 2019"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] Who won the ICC world cup 2019 [/INST] The ICC World Cup 2019 was won by England. They defeated New Zealand in the final match, which was played at the Melbourne Cricket Ground in Australia. The match was a close one, with England winning by 24 runs. This victory marked England's first ICC World Cup win since 2007. The tournament was held from 17 October to 1 November 2019, and it was the 11th edition of the ICC World Cup. The tournament was played in 10 countries, including England, Australia, South Africa, and India. The ICC World Cup is the most prestigious cricket tournament in the world, and it is played every four years. The tournament features the best cricket teams from around the world, and it is a chance for these teams to compete for the coveted ICC World Cup trophy. The tournament is played over a period of 51 days, and it includes a total of 11 matches, including the final match. The tournament is a major event in the cricket calendar, and it is watched by millions 